In [7]:
import os
import re
import numpy as np
import pandas as pd
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
from pathlib import Path

Define Paths

In [8]:
processed_data_dir = "../data/processed"
output_data_dir = "../data/outputs"

os.makedirs(output_data_dir, exist_ok=True)

Load/Align Rasters Helper Function

In [4]:
def load_and_align_rasters(file_list, reference_raster=None):
    """
    Loads a list of rasters and aligns them to a common resolution/extent.
    If reference_raster is provided, aligns all to that raster.
    Returns an xarray.Dataset with all rasters stacked as variables.
    """
    rasters = []
    names = []

    for file in file_list:
        arr = rxr.open_rasterio(file, masked=True).squeeze()
        names.append(os.path.splitext(os.path.basename(file))[0])

        # Reproject/align if reference is given
        if reference_raster is not None:
            arr = arr.rio.reproject_match(reference_raster)

        rasters.append(arr)

    # Stack into dataset
    ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])
    return ds

Severity Classification Function

In [5]:
def classify_severity(dnbr):
    if dnbr < 0.1:
        return "Unburned"
    elif dnbr < 0.27:
        return "Low"
    elif dnbr < 0.44:
        return "Moderate"
    else:
        return "High"

Process All Fires

In [6]:
all_fire_dfs = []

# Detect fires based on available files
fires = sorted(set(os.path.basename(f).split("_")[0] for f in glob(os.path.join(processed_data_dir, "*.tif"))))

for fire_name in fires:
    print(f"Processing {fire_name} fire ...")

    fire_files = glob(os.path.join(processed_data_dir, f"{fire_name}_*.tif"))
    dnbr_path = os.path.join(processed_data_dir, f"{fire_name}_dNBR.tif")

    if not os.path.exists(dnbr_path):
        print(f"Skipping {fire_name} — no dNBR available.")
        continue

    try:
        reference = rxr.open_rasterio(dnbr_path, masked=True).squeeze()
        fire_ds = load_and_align_rasters(fire_files, reference_raster=reference)

        # Mask no-data areas based on dNBR
        fire_ds = fire_ds.where(~np.isnan(fire_ds[f"{fire_name}_dNBR"]), drop=True)

        # Convert to dataframe
        df = fire_ds.to_dataframe().reset_index()
        df = df.dropna()
        df["fire_name"] = fire_name
        df["severity"] = df[f"{fire_name}_dNBR"].apply(classify_severity)

        # Save individual file
        fire_output = os.path.join(output_data_dir, f"{fire_name}_dataset.parquet")
        df.to_parquet(fire_output, index=False)
        print(f"Saved {fire_name} dataset with {len(df)} pixels → {fire_output}")

        all_fire_dfs.append(df)

    except Exception as e:
        print(f"Error processing {fire_name}: {e}")



Processing Bootleg fire ...
Skipping Bootleg — no dNBR available.
Processing Caldor fire ...


C:\Users\John Waugh\AppData\Local\Temp\ipykernel_17020\1609434101.py:21: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])


Saved Caldor dataset with 5517951 pixels → ../data/outputs\Caldor_dataset.parquet
Processing Camp fire ...


C:\Users\John Waugh\AppData\Local\Temp\ipykernel_17020\1609434101.py:21: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])


Saved Camp dataset with 4412816 pixels → ../data/outputs\Camp_dataset.parquet
Processing Dixie fire ...


C:\Users\John Waugh\AppData\Local\Temp\ipykernel_17020\1609434101.py:21: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])


Saved Dixie dataset with 5625635 pixels → ../data/outputs\Dixie_dataset.parquet
Processing Troublesome fire ...


C:\Users\John Waugh\AppData\Local\Temp\ipykernel_17020\1609434101.py:21: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])


Saved Troublesome dataset with 5576320 pixels → ../data/outputs\Troublesome_dataset.parquet


Combine Fires

In [ ]:
fire_datasets = list(Path(output_data_dir).glob('*_dataset.parquet'))

combined_list = []

for file in fire_datasets:
    fire_name = file.stem.replace('_dataset', '')
    
    df = pd.read_parquet(file)
    
    df.columns = [re.sub(f"^{fire_name}_", "", c) for c in df.columns]
    
    df = df.rename(columns={
        'x': 'longitude',
        'y': 'latitude',
        'veg_indices': 'NDVI'
    })
    
    if 'fire_name' not in df.columns:
        df['fire_name'] = fire_name
    else:
        df['fire_name'] = df['fire_name'].fillna(fire_name)
    
    expected_cols = ['latitude', 'longitude', 'dNBR', 'SPI', 'VCI', 'NDVI', 'severity', 'fire_name']
    available_cols = [c for c in expected_cols if c in df.columns]
    df = df[available_cols]
    
    combined_list.append(df)

combined_df = pd.concat(combined_list, ignore_index=True)

combined_df.to_parquet('combined_dataset.parquet', index=False)

print(f"Combined dataset created: {len(combined_df):,} rows across {len(fire_datasets)} events")
print("Columns:", list(combined_df.columns))

combined_df.head()
combined_df['fire_name'].value_counts()
combined_df['severity'].value_counts()

Combined dataset created: 42,265,444 rows across 5 events
Columns: ['latitude', 'longitude', 'dNBR', 'SPI', 'VCI', 'NDVI', 'severity', 'fire_name']


severity
Unburned    39231620
Low          2135624
Moderate      496776
High          401424
Name: count, dtype: int64